<a href="https://colab.research.google.com/github/YounSooKimTech/Joy_Automation/blob/main/2023_11_12_predoc_org_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install beautifulsoup4 requests

# Web scrap of the following page
https://predoc.org/opportunities

In [15]:
from bs4 import BeautifulSoup
import requests

url = "https://predoc.org/opportunities"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'predoc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [16]:
institutions = []
fields = []
deadlines = []
researchers =[]

copy_divs = soup.find_all('div', class_='copy')

for copy_div in copy_divs:
  temp_text = copy_div.get_text()

  temp_researcher = temp_text[temp_text.find("Researcher"):].split("\n")[0].split(":")[-1].strip()
  researchers.append(temp_researcher)

  temp_field = temp_text[temp_text.find("Field"):].split("\n")[0].split(":")[-1].strip()
  if temp_field.lower() == "field":
    temp_field = ""
  fields.append(temp_field)

  temp_institution = temp_text[temp_text.find("Institution"):].split("\n")[0].split(":")[-1].strip()
  institutions.append(temp_institution)

  temp_dead = temp_text[temp_text.find("Deadline"):].split("\n")[0]
  temp_dead_split = temp_dead.split(":")
  if len(temp_dead_split)>1:
    deadlines.append(temp_dead_split[1])
  else:
    deadlines.append("")


In [17]:
copy_divs = soup.find_all('div', class_='copy')
h_links = []

for copy_div in copy_divs:
  a_tag = copy_div.find("a")
  h_links.append(a_tag["href"])

In [20]:
import pandas as pd

df_op = pd.DataFrame({"Deadline_text": deadlines,
                      "Institution":institutions,
                      "Researcher": researchers,
                      "Field":fields,
                      "Links":h_links})
df_op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Deadline_text  172 non-null    object
 1   Institution    172 non-null    object
 2   Researcher     172 non-null    object
 3   Field          172 non-null    object
 4   Links          172 non-null    object
dtypes: object(5)
memory usage: 6.8+ KB


In [21]:
import numpy as np

# change the deadline into the consistent form
date_pattern = r"(?i)((?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2})"

df_op['Deadline'] = df_op['Deadline_text'].str.extract(date_pattern, expand=False)
df_op.loc[df_op['Deadline'].isna() & df_op['Deadline_text'].str.contains(r'(?i)rolling'), 'Deadline'] = 'Rolling'
df_op.loc[df_op['Deadline'].isna() & df_op['Deadline_text'].str.contains(r'(?i)ASAP'), 'Deadline'] = 'ASAP'

In [22]:
df_op["Deadline"].value_counts(dropna=False)

Rolling         84
NaN             25
October 15      12
October 16       7
January 5        4
September 17     4
ASAP             4
September 15     2
December 15      2
October 6        2
November 10      2
November 15      2
October 20       1
August 14        1
March 6          1
April 30         1
April 14         1
June 15          1
June 16          1
July 20          1
August 31        1
September 1      1
January 20       1
August 28        1
October 31       1
September 20     1
December 31      1
November 1       1
October 1        1
December 1       1
October 4        1
February 1       1
October 30       1
May 31           1
Name: Deadline, dtype: int64

In [29]:
import pandas as pd
from datetime import datetime

def convert_date_format(date_text):
    if date_text not in ['ASAP', 'Rolling'] and not pd.isna(date_text):
        date_obj = datetime.strptime(date_text, "%B %d")
        formatted_date = date_obj.strftime("%m-%d")
        return formatted_date
    elif date_text in ['ASAP', 'Rolling']:
        return date_text
    else:
        return date_text

df_op["Deadline"] = df_op["Deadline"].apply(convert_date_format)

df_op.head(2)


,Deadline_text,Institution,Researcher,Field,Links,Deadline
0,"January 5, 2024","Northwestern University, Kellogg School of Man...",Ameet Morjaria,"Economic Development, Political Economy, Organ...",https://bit.ly/40fe8cz,01-05
1,"January 5, 2024","Northwestern University, Kellogg School of Man...","Chethana Achar, Jake Teeny","Behavioral Marketing, Consumer Behavior",https://bit.ly/46dKaa2,01-05


In [31]:
df_op = df_op[df_op["Deadline"].str.startswith(("11", "12", "01", "02", "03")) |
 (df_op["Deadline"].isin(["Rolling", "ASAP"]))]

In [33]:
df_op = df_op[["Deadline", "Links", "Institution", "Researcher", "Field", "Deadline_text"]]
df_op


In [34]:
import pandas as pd
from datetime import datetime

today_date = datetime.today().strftime('%Y-%m-%d')
file_name = f"predoc_webscrapping_{today_date}.xlsx"
df_op.to_excel(file_name, index=False)


In [9]:
#df_op.dropna(subset=["Deadline"], inplace=True)
#df_op.info()

# Temp codes

## Web scrap of the following page:

https://predoc.org/ra-opportunities

In [12]:
# Due to the characteristics in the page
# I used text extraction with index for the information
# maybe, using text[index:] can be a better way to extract information I guess

In [13]:
from bs4 import BeautifulSoup
import requests

url = "https://predoc.org/ra-opportunities"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'predoc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [14]:
div_tags = soup.find_all("div", class_ = "body-copy")

h_links = []

for div_tag in div_tags:
  li_tags = div_tag.find_all("li")
  for li_tag in li_tags:
    a_tag = li_tag.find("a")
    h_links.append(a_tag["href"])

h_links

TypeError: ignored

In [ ]:
institutions = []
fields = []
deadlines = []
researchers =[]

div_tags = soup.find_all("div", class_ = "body-copy")

for div_tag in div_tags:
  li_tags = div_tag.find_all("li")
  for li_tag in li_tags:

    temp_text = li_tag.get_text()

    temp_inst = temp_text[temp_text.find("Institution"):].split(":")[1].split("\n")[0].strip()
    institutions.append(temp_inst)

    temp_field = temp_text[temp_text.find("Fields"):].split(":")[1].split("\n")[0].strip().replace(".", "")
    fields.append(temp_field)

    temp_deadline = temp_text[temp_text.find("Deadline"):].split(":")[1].split("\n")[0].strip()
    deadlines.append(temp_deadline)

    temp_researcher = temp_text[temp_text.find("Researcher"):].split("\n")[0].split(":")[-1].strip()
    if len(temp_researcher)<3:
      temp_researcher = ""
    researchers.append(temp_researcher)



In [ ]:
print(len(institutions), len(h_links))

In [ ]:
# make it as a dataframe

import pandas as pd

df_ra = pd.DataFrame({"Deadlines": deadlines,
                      "Institution":institutions,
                      "Researcher": researchers,
                      "Field":fields,
                      "Links":h_links})


# change the deadline into the consistent form
date_pattern = r"(?i)((?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2})"

df_ra['Deadline'] = df_ra['Deadlines'].str.extract(date_pattern, expand=False)
df_ra.loc[df_ra['Deadline'].isna() & df_ra['Deadlines'].str.contains(r'(?i)rolling'), 'Deadline'] = 'Rolling'
df_ra.loc[df_ra['Deadline'].isna() & df_ra['Deadlines'].str.contains(r'(?i)ASAP'), 'Deadline'] = 'ASAP'

In [ ]:
df_ra.sample(3)

## Sort the data

In [ ]:
import datetime

def convert_deadline(deadline):
  if deadline == "ASAP":
    return pd.to_datetime("2099-12-31")
  elif deadline == "Rolling":
    return pd.to_datetime("2098-12-31")
  elif pd.isna(deadline):
    return pd.to_datetime("2099-12-31")
  else:
    return pd.to_datetime(f"{datetime.date.today().year} {deadline}", format = "%Y %B %d")

df_ra["deadline_index"] = df_ra["Deadline"].apply(convert_deadline)

today = datetime.datetime.now().date()
df_ra = df_ra[df_ra["deadline_index"] >= pd.to_datetime(today)]

# sort the data by date
df_ra = df_ra.sort_values(by = "deadline_index", ascending=True)

# change the order
df_ra = df_ra[["Deadline", "Institution", "Researcher", "Field", "Links", "Deadlines"]]

In [ ]:
df_ra.head(2)

In [ ]:
df_op["deadline_index"] = df_op["Deadline"].apply(convert_deadline)


today = datetime.datetime.now().date()
df_op = df_op[df_op["deadline_index"] >= pd.to_datetime(today)]

# sort the data by date
df_op = df_op.sort_values(by = "deadline_index", ascending=True)

# change the order
df_op = df_op[["Deadline", "Institution", "Researcher", "Field", "Links", "Deadlines"]]

In [ ]:
df_op.head(2)

## Save the output as Exce lfile

In [ ]:
import pandas as pd

with pd.ExcelWriter("output.xlsx") as writer:

    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    df_op.to_excel(writer, sheet_name="Opportunities page", index=False)
    df_ra.to_excel(writer, sheet_name="RA Opportunities page", index=False)


In [ ]:
from google.colab import files
files.download("output.xlsx")

## Extra Codes
Opportunities page (with html header)

In [ ]:
# In case of using html headers

from bs4 import BeautifulSoup
import requests

url = "https://predoc.org/opportunities"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')


In [ ]:
# Deadlines
# nested under div > p > strong > Deadline

copy_divs = soup.find_all('div', class_='copy')

deadlines = []

for copy_div in copy_divs:
    p_tags = copy_div.find_all("p")
    for p_tag in p_tags:
        strong_tags = p_tag.find_all('strong')
        for strong_tag in strong_tags:
          # print(strong_tag.get_text(), strong_tag.next_sibling, "\n")
          if "Deadline" in strong_tag.get_text():
            # print(strong_tag.next_sibling)
            next_sibling = strong_tag.next_sibling
            if next_sibling:
              deadlines.append(next_sibling.strip())
            else:
              deadlines.append("")  # Add an empty string if "researchers" is not found

fields = []

for copy_div in copy_divs:
    p_tags = copy_div.find_all("p")
    for p_tag in p_tags:
        strong_tags = p_tag.find_all('strong')
        for strong_tag in strong_tags:
          # print(strong_tag.get_text(), strong_tag.next_sibling, "\n")
          if "Field" in strong_tag.get_text():
            # print(strong_tag.next_sibling)
            next_sibling = strong_tag.next_sibling
            if next_sibling:
              fields.append(next_sibling.strip().strip(":"))
            else:
              fields.append("")  # Add an empty string if "researchers" is not found


print(len(fields))


researchers = []
for copy_div in copy_divs:
    p_tags = copy_div.find_all("p")
    for p_tag in p_tags:
      #print(p_tag.get_text(), "\n")
      if "Researcher" in p_tag.get_text() or "Researcher(s)" in p_tag.get_text():
        strong_tags = p_tag.find_all('strong')
        for strong_tag in strong_tags:
          if "Researcher" in strong_tag.get_text() or "Researcher(s)" in strong_tag.get_text() :
              next_sibling = strong_tag.next_sibling
              researchers.append(next_sibling.strip().strip(":").strip("()"))
      else:
        researchers.append("None")


links = []
a_tags = soup.select('div.copy a.general-link')
for a_tag in a_tags:
    # print(a_tag["href"])
    links.append(a_tag["href"])
